# Basics

There are two ways to create agents 🤖:

1. Using the Azure AI Agent Service SDK. **This is the recommended approach**.
2. Using the OpenAI SDK. In this case they are not called agents, but assistants.

The code below will import some Python classes and functions, will read the .env file, and get ready the environment.
In addition it will define the function pprint, which will be used in this workshop to improve the readability of the objects created.

In [8]:
import os
from pprint import pp as pp

from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Load environment variables from .env file
load_dotenv(override=True)

def pprint(obj):
    pp(obj.as_dict() if hasattr(obj, "as_dict") else obj, width=100)

# Print the environments we will be using.
print(f"PROJECT_CONNECTION_STRING: {os.getenv('PROJECT_CONNECTION_STRING')}")
print(f"BING_CONNECTION_NAME: {os.getenv('BING_CONNECTION_NAME')}")  

PROJECT_CONNECTION_STRING: 576a663a-07b7-4dd2-82d5-1347867b172a.workspace.eastus.api.azureml.ms;698d9beb-2466-404e-ae80-79d2faa013b1;nbm-eastus-rg;nbm-eastus-ai1
BING_CONNECTION_NAME: bing_grounding


Get the project reference using the current authenticated user and the connection string to the project where the agents will be created. 

In [9]:
project = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## Creating Agents using Agent Service SDK

Now that we are connected to the project, let's create a very simple agent using **GPT-4o-mini** model as the agent _"brain"_. We are also providing instructions on how the agent should behave, adding a description and custom metadata fields. These metadata fields can be used to add custom information to find agents later for scenario with several agents.

In [10]:
portfolio_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Portfolio Guidance",
    instructions="You are a financial agent, providing portfolio guidance for users.",
    description="This agent was created to provide guidance for our users.",
    metadata= {
        "department": "finance",
        "owner": "jdoe"
    }
)
pprint(portfolio_agent)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting 

Every agent is unique and has an id, which starts with the string _asst_. This id and can be used to retrieve the agent later.

Let's create a second agent, but now for the marketing team.

In [ ]:
branding_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Branding Guidance",
    instructions="You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos.",
    description="This agent was created to provide guidance for our users.",
    metadata={
        "department": "finance",
        "owner": "jdoe"
    }
)
pprint(branding_agent)

Using the project object it is possible to list all agents in the project, delete an agent or update an agent.

Right now we should have two agents created in the project.

In [ ]:
pprint(project.agents.list_agents())

Let's improve the instructions we provided for the portfolio agent.

In [ ]:
portfolio_agent = project.agents.update_agent(portfolio_agent.id, instructions="You are a financial agent, providing portfolio guidance for users. Avoid using too technical terms, and be clear about the risks.")
pprint(portfolio_agent)

### Chatting with the Agent

In order to chat with an agent, we need to create a **thread**. A thread is a **container** for messages, and there are two types of messages: messages created by the user or created by an agent.

**Important!**
1. If you don't save the thread id, you won't be able to recover the thread later. Keep this in mind, and save it to another database.
2. A thread is not tied to an agent. **The same thread can be used by several agents**.
3. Unless deleted manually, threads are kept forever.

Jane wants to get information on how to invest her money, so let's create a thread for Jane.
You can also add metadata for threads, and in this case we are adding the user id from Microsoft Entra.

In [ ]:
thread_jane = project.agents.create_thread(
    metadata= {
        "entraUserId": "a1ddfd30-8420-4a8d-b155-4b5f05998d69"
    }
)
pprint(thread_jane)

Let's add a message to the thread we created.

In [ ]:
message = project.agents.create_message(
    thread_id=thread_jane.id,
    role="user",
    content="What is the best option to invest my money?",
)
pprint(message)

Let's print all messages in the thread.

In [ ]:
pprint(project.agents.list_messages(thread_jane.id))

At this point, we created a thread containing only one message. As you can see, the thread or message are not linked to any assistant. You can validate that looking the **run_id** and **assistant_id** properties for messages where the role is **user**.

Now, it's time to ask the agent to reason over the message the user sent and generate a reply. This is represented by a run object. 

The execution of the agent is **asynchronous** and we need to poll if its execution was completed. The function **create_and_process_run** abstracts all this process.

In [ ]:
pprint(project.agents.create_and_process_run(thread_id=thread_jane.id, assistant_id=portfolio_agent.id)) 

Now, let's list the message that was created and added to the thread. You will see that the agent added a new message to the list.

In [ ]:
pprint(project.agents.list_messages(thread_jane.id))

## Creating Agents using OpenAI SDK

Another way of creating agents is using OpenAI SDK. In this case, they are called **assistants**. 

To create one, we need an Azure OpenAI deployment reference, which can be acquired using the following code.

In [12]:
openai_client = project.inference.get_azure_openai_client(
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

Now, let's create an assistant to help users about branding questions using this client.

In [ ]:
branding_agent = openai_client.beta.assistants.create(
    model="gpt-4o-mini",
    name="Branding Guidance",
    instructions="You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos."
)
pprint(branding_agent)

The same way we did using Agent Service SDK, we can use the OpenAI SDK to list, update or delete assistants.

In [ ]:
pprint(openai_client.beta.assistants.list())

Maria has some questions about how the company logo can be used. We will create using a single call a thread with a list of messages. Previously we created this in two differente calls.

In [ ]:
thread_maria = openai_client.beta.threads.create(
    messages= [
        {
            "role": "user",
            "content": "I need to add the company logo side by side with another company logo. Is there any guidance about it?",         
        }
    ]
)
pprint(thread_maria)

Execute the assistant over the thread.

In [ ]:
run = openai_client.beta.threads.runs.create(
    thread_id=thread_maria.id,
    assistant_id=branding_agent.id,
)
pprint(run)

Print the message created by the assistant.

In [ ]:
pprint(openai_client.beta.threads.messages.list(thread_id=thread_maria.id))

As you can see using Agent Service SDK or OpenAI SDK the experience today is almost the same, however the recomendation is to use the Agent Service SDK.


You have reached the end of this lab. 👏